In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns

In [2]:
plt.rcParams["figure.figsize"]=[15,5]

In [3]:
raw_ld=pd.read_csv('6thHour.csv', usecols=['timestamp', 'data.ax', 'data.az'])
raw_ld.sort_values(by=['timestamp'], inplace=True)
raw_ld.reset_index(inplace=True)
raw_ld.dropna(axis=0, inplace=True)
raw_ld.timestamp=pd.to_datetime(raw_ld.timestamp)

In [4]:
#Load LD data and plot
raw_ld["acc"] = ( raw_ld["data.ax"]**2 + raw_ld["data.az"]**2 ) ** 0.5
acc=raw_ld[["timestamp", "acc"]]
acc.set_index('timestamp')
test_ld=acc
test_ld['acc_raw']=test_ld['acc']
test_ld['acc']=pd.Series.to_frame(test_ld.acc.rolling(100, center=True).std())

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [5]:
# test_ld['timestamp'] =  pd.to_datetime(test_ld['timestamp'])

In [6]:
timethresh=12
test_ld.insert(2,'state',0)

In [7]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [8]:
plt.plot(test_ld['timestamp'], test_ld['acc'])

In [9]:
#Get boards from Loader
board=scipy.signal.find_peaks(test_ld.acc, height=(0.01), distance=timethresh*72, width=1)
differ=np.diff(board[0])
differ=differ.tolist()

# boards=[]
# boards.append(board[0][0])
# sumdiffer=0
# for i, x in enumerate(board[0]):
#     try:
#         sumdiffer+=differ[i]
#         if sumdiffer>timethresh*100:
#             boards.append(board[0][i])
#             test_ld.at[x, 'state']=1
#             sumdiffer=0
#         if differ[i] < timethresh*100:
#             test_ld.at[x, 'state']=2
#     except:
#         continue
for x in board[0]:
    test_ld.at[x, 'state'] = 1

ld_boards=test_ld[test_ld['state']==1]
print('Boards detected in LD:', ld_boards.shape[0])

Boards detected in LD: 34


In [33]:
fig, ax1 = plt.subplots()


# ax1.plot(test_ld['timestamp'], test_ld['acc_raw'], 'green')
ax1.plot(test_ld['timestamp'], np.ones(test_ld.shape[0])*0.01, 'green')
ax1.plot(test_ld['timestamp'], test_ld['acc'], 'orange')

# ax1.plot(test_ld['timestamp'], np.ones(test_ld.shape[0]*0.01), 'green')
# ax1.xaxis.set_major_locator(mdates.DateFormatter('%H:%M'))

ax1.set_xlabel('Time')

ax2 = ax1.twinx()
ax2.stem(ld_boards['timestamp'], ld_boards['state'], 'b:')

plt.title('Board Detections')

Text(0.5, 1.0, 'Board Detections')

In [ ]:
loading_delays_rawdf=pd.DataFrame({"sample_number":board[0], "working_time":board[1]['widths']/72})


In [ ]:
LD_events=pd.DataFrame({"timestamp":test_ld.iloc[loading_delays_rawdf.sample_number].timestamp, "event":1, "working_time":(loading_delays_rawdf.working_time).tolist()})

In [ ]:
LD_events.index=LD_events.timestamp
LD_events.drop('timestamp', axis=1, inplace=True)
LD_events['energy']=float('nan')

In [ ]:
LD_events

In [ ]:
sns.distplot(LD_events.working_time)
plt.title('Loader Loading Times Histogram')

In [ ]:
print('LD loading time mode: ', LD_events.working_time.mode().mean())